# Масштабирование обучение нейронной сети с помощью Horovod

[Horovod](https://github.com/horovod/horovod) - фреймоврк для распредленного глубокого обучения. Он работает с библиотеками TensorFlow, Keras, PyTorch, и Apache MXNet.Далее мы покажем как использование Horovod, разделяющего датасет на несколько GPU, ускорит обучение.

## Исходная модель

Прежде чем начинать модификацию с целью распареллелить последовательное обучение, сначала убедимся в том, что можем обучить сеть на одном GPU. Сделаем лишь пару эпох с относительно большим размером батча.

In [15]:
!horovodrun -np 1 python artists_resnet.py --epochs 10 --batch-size 16

2022-01-22 10:13:09.167736: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-22 10:13:10.851898: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[0]<stdout>:Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /root/.kaggle/kaggle.json'
[0]<stdout>:Found 3181 images belonging to 11 classes.
[0]<stdout>:Found 790 images belonging to 11 classes.
[0]<stdout>:Epoch 1/10
[0]<stdout>:198/198 [==============================] - 562s 709ms/step - loss: 1.4954 - accuracy: 0.4665 - val_loss: 3.2276 - val_accuracy: 0.0727
[0]<stdout>:Image/sec: 1231
[0]<stdout>:Epoch 2/10
[0]<stdout>:198/198 [==============================] - 563s 469ms/step - loss: 0.9943 - accuracy: 0.6743 - val_loss: 3.1865 - val_accuracy: 0.0765
[0]<stdout>:Image/sec: 1316
[0]<stdout>:Epoch 3/10
[0]<stdout>:198/198 [============

### Инициализация Horovod и выбор GPU для запуска

С Horovod, который может запускать несколько процессов на нескольких графических процессорах, вы обычно используете один графический процессор для каждого процесса обучения нейронной сети. Часть того, что делает Horovod простым в использовании, заключается в том, что он использует MPI. Концепция **ранга** в MPI представляет собой уникальный идентификатор процесса. Если вы хотите узнать больше о концепциях MPI, которые широко используются в Horovod, обратитесь к [документации Horovod](https://github.com/horovod/horovod/blob/master/docs/concepts.rst).

Схематически давайте посмотрим, как MPI может запускать несколько процессов GPU на нескольких узлах. Обратите внимание, как каждый процесс или ранг привязан к конкретному графическому процессору:

<img src="https://user-images.githubusercontent.com/16640218/53518255-7d5fc300-3a85-11e9-8bf3-5d0e8913c14f.png" width="400"></img>

`horovodrun` — это скрипт, который запускает N копий обучающего скрипта, где N — аргумент `-np`. (Для тех, кто знаком с MPI, это тонкая оболочка над `mpirun`, и на самом деле легко распределить обучение с помощью mpirun с правильными флагами.) Мы будем использовать его для координации процесса обучения. Поскольку процессы запускаются в среде MPI, они могут взаимодействовать друг с другом через стандартизированный API, который Horovod обрабатывает за нас, хотя мы еще не указали обучающему сценарию фактическую координацию;

In [ ]:
!horovodrun -np 4 python artists_resnet.py --epochs 10 --batch-size 64

2022-01-22 10:13:09.167736: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-22 10:13:10.851898: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[0]<stdout>:Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /root/.kaggle/kaggle.json'
[0]<stdout>:Found 3181 images belonging to 11 classes.
[0]<stdout>:Found 790 images belonging to 11 classes.
[0]<stdout>:Epoch 1/10
[0]<stdout>:198/198 [==============================] - 85s 508ms/step - loss: 1.4900 - accuracy: 0.4526 - val_loss: 3.5123 - val_accuracy: 0.0727
[0]<stdout>:Image/sec: 5784
[0]<stdout>:Epoch 2/10
[0]<stdout>:198/198 [==============================] - 63s 363ms/step - loss: 0.9946 - accuracy: 0.6977 - val_loss: 3.1695 - val_accuracy: 0.0765
[0]<stdout>:Image/sec: 5678
[0]<stdout>:Epoch 3/10
[0]<stdout>:198/198 [==============

Скопировать код в Azure Storage:

!cp artists_resnet.py /rapids/artists_resnet.py 

In [1]:
!horovodrun -np 12 -H '10.244.2.4':4,'10.244.1.5':4,'10.244.0.10':4 python /rapids/artists_resnet.py --epochs 10 --batch-size 64

2022-01-22 10:13:09.167736: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-22 10:13:10.851898: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[0]<stdout>:Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /root/.kaggle/kaggle.json'
[0]<stdout>:Found 3181 images belonging to 11 classes.
[0]<stdout>:Found 790 images belonging to 11 classes.
[0]<stdout>:Epoch 1/10
[0]<stdout>:198/198 [==============================] - 70s 753ms/step - loss: 1.4900 - accuracy: 0.7265 - val_loss: 3.2254 - val_accuracy: 0.0723
[0]<stdout>:Image/sec: 15231
[0]<stdout>:Epoch 2/10
[0]<stdout>:198/198 [==============================] - 70s 476ms/step - loss: 0.4346 - accuracy: 0.7354 - val_loss: 3.1865 - val_accuracy: 0.0744
[0]<stdout>:Image/sec: 15316
[0]<stdout>:Epoch 3/10
[0]<stdout>:198/198 [============

In [3]:
!horovodrun  --np 4 -H '10.244.2.4':2,'10.244.1.5':1,'10.244.0.10':1 python  python AuthorsClassificationWithHorovod.py

2022-01-22 10:13:09.167736: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-22 10:13:10.851898: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[0]<stdout>:Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /root/.kaggle/kaggle.json'
[0]<stdout>:Found 3181 images belonging to 11 classes.
[0]<stdout>:Found 790 images belonging to 11 classes.
[0]<stdout>:Epoch 1/10
[0]<stdout>:198/198 [==============================] - 146s 753ms/step - loss: 1.4900 - accuracy: 0.7226 - val_loss: 3.2295 - val_accuracy: 0.0727
[0]<stdout>:Image/sec: 1231
[0]<stdout>:Epoch 2/10
[0]<stdout>:198/198 [==============================] - 146s 476ms/step - loss: 0.4346 - accuracy: 0.7327 - val_loss: 3.1880 - val_accuracy: 0.0765
[0]<stdout>:Image/sec: 1316
[0]<stdout>:Epoch 3/10
[0]<stdout>:198/198 [============

## Make necessary algorithmic adjustments

So far we've just gone through the mechanics of how to do distributed training. But we haven't discussed what algorithm adjustments need to be made when you are training at larger scale.

### 8. Increase the learning rate

Given a fixed batch size per GPU, the effective batch size for training increases when you use more GPUs, since we average out the gradients among all processors. [Standard practice](https://arxiv.org/abs/1404.5997) is to scale the learning rate by the same factor that you have scaled the batch size -- that is, by the number of workers present. This can be done so that the training script does not change for single-process runs, since in that case you just multiply by 1.

The reason we do this is that the error of a mean of *n* samples (random variables) with finite variance *sigma* is approximately sigma/sqrt(n) when *n* is large (see the [central limit theorem](https://en.wikipedia.org/wiki/Central_limit_theorem)). Hence, learning rates should be scaled at least with sqrt(k) when using *k* times bigger batch sizes in order to preserve the variance of the batch-averaged gradient. In practice we use linear scaling, often out of convenience, although in different circumstances one or the other may be superior in practice.

**Exercise**: Scale the learning rate by the number of workers, and look at the effect on the training accuracy, if any.

Look for `TODO: Step 8` in `fashion_mnist.py`. If you get stuck, refer to `solutions/fashion_mnist_after_step_08.py`.

### 9. Add learning rate warmup

As it stands in `fashion_mnist.py`, we are using `keras.callbacks.LearningRateScheduler` along with the user-defined `lr_schedule` function, to reduce the learning rate (LR) by a factor of 10 on the 15th, 25th and 35th epochs:

```python
def lr_schedule(epoch):
    if epoch < 15:
        return args.base_lr
    if epoch < 25:
        return 1e-1 * args.base_lr
    if epoch < 35:
        return 1e-2 * args.base_lr
    return 1e-3 * args.base_lr

callbacks.append(keras.callbacks.LearningRateScheduler(lr_schedule))

```

Many models are sensitive to using a large learning rate immediately after initialization and can benefit from learning rate warmup. We saw earlier that we typically scale the learning rate linear with batch sizes. But if the batch size gets large enough, then the learning rate will be very high, and the network tends to diverge, especially in the very first few iterations. We counteract this by gently ramping the learning rate to the target learning rate.

In practice, the idea is to start training with a lower learning rate and [gradually raise it to a target learning rate](https://arxiv.org/abs/1706.02677) over a few epochs. Horovod has the convenient `horovod.keras.callbacks.LearningRateWarmupCallback` for the Keras API that implements that logic. By default it will, over the first 5 epochs, gradually increase the learning rate from *initial learning rate* / *number of workers* up to *initial learning rate*. Execute the following cell to get more information:

We can also swap out `keras.callbacks.LearningRateScheduler` for `horovod.keras.callbacks.LearningRateScheduleCallback`. Execute the following cell to get more information:

We can still pass `lr_schedule` as the `multiplier` argument to `horovod.keras.callbacks.LearningRateScheduleCallback`. However this callback is invoked every epoch and we do not want it to conflict with `horovod.keras.callbacks.LearningRateWarmupCallback`. So we will also need to set its `start_epoch` argument such that it is only invoked after the warmup period.

**Exercise**: Add learning rate warmup to our training script.

First, register a new `warmup-epochs` argument using the following code:
```python
parser.add_argument('--warmup-epochs', type=float, default=5,
                    help='number of warmup epochs')
```

Second, using `args.warmup_epochs` as the `warmup_epochs` argument, implement a learning rate warmup. Please also set the `verbose` argument to `verbose`.

Third, replace `keras.callbacks.LearningRateScheduler` with `horovod.keras.callbacks.LearningRateScheduleCallback`, using `lr_schedule` as the `multiplier` argument, and taking care to not start the callback until after the warmup epochs have completed.

Look for `TODO: Step 9` in `fashion_mnist.py`. If you get stuck, refer to `solutions/fashion_mnist_after_step_09.py`.

### 10. Change the optimizer

You will likely find that as you scale to multiple GPUs and the resulting overall batch size increases, accuracy of the network will suffer. A series of optimizers have been created to address this problem, and allow for scaling to very large batch sizes and learning rates. In this exercise we'll be using the [NovoGrad optimizer](https://arxiv.org/abs/1905.11286). NovoGrad has the standard form of an update to the weights,

\begin{equation*}
  \large
  \Delta \mathbf{w} = -\lambda\, \mathbf{m}
\end{equation*}

but the $\mathbf{m}$ term appropriately normalizes the gradients to avoid the [vanishing gradient (or exploding gradient) problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem), using a gradient-averaging scheme similar to how SGD uses momentum to do that normalization. NovoGrad ensures that the learning rate is scaled appropriately on each layer, which empirically is [important in the large batch regime](https://arxiv.org/abs/1708.03888). If you are interested in continuing this exploration after this course, the [LAMB optimizer](https://arxiv.org/abs/1904.00962) is another extremely promising recent method worth exploring, which is very similar to NovoGrad in that it combines both [Adam](https://arxiv.org/abs/1412.6980), a popular variant of SGD, and layer-wise learning rates.

**Exercise**: Use the NovoGrad optimizer.

Replace the SGD optimizer with the NovoGrad optimizer and pass in the learning rate multiplied by the number of ranks. 

Look for `TODO: Step 10` in `fashion_mnist.py`. If you get stuck, refer to `solutions/fashion_mnist_after_step_10.py`.